# assignment 7

In [1]:
# from numpy import power, arange, asarray, argsort, abs
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import KFold

получаем данные

In [2]:
newsgrop = fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])

No handlers could be found for logger "sklearn.datasets.twenty_newsgroups"


записываем данные в переменную признаков `X` и целевую `y`

In [ ]:
X = newsgrop.data
y = newsgrop.target

преобразуем обучающую выборку

In [4]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X)

создаем генератор разбиений

In [5]:
cv = KFold(y.size, n_folds=5, shuffle=True, random_state=241)

задаем коэффициент C, который будем подбирать
$$ C \in [10^{-5}, 10^{-4}, ..., 10^4, 10^5] $$

In [6]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}

создаем классификатор опорных векторов

In [7]:
clf = SVC(kernel='linear', random_state=241)

подбираем параметр `C`

In [ ]:
gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv)

In [8]:
# gs = GridSearchCV(clf, grid, cv=5)
gs.fit(X_train, y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=1786, n_folds=5, shuffle=True, random_state=241),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

находим лучший параметр `C`

In [9]:
best_param = gs.best_params_

создаем и обучаем новый классификатор с оптимальным параметром C

In [10]:
clf_opt = SVC(C=best_param['C'], kernel='linear', random_state=241)
clf_opt.fit(X_train, y)

SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [29]:
# находим веса
dens_f = clf_opt.coef_.todense()
dens_f

matrix([[ 0.27365782, -0.10705497,  0.        , ...,  0.01994628,
          0.05950164, -0.00301381]])

In [37]:
# получаем индексы слов с наибольшим весом
import scipy.sparse
coef = scipy.sparse.coo_matrix(clf_opt.coef_)
import heapq
max_weights = heapq.nlargest(10, zip(coef.col, coef.data), lambda _: abs(_[1]))
max_weight = []
for i in range(len(max_weights)):
    max_weight.append(max_weights[i][0])

In [38]:
max_weight

[24019, 12871, 5088, 5093, 17802, 23673, 21850, 5776, 15606, 18430]

In [18]:
# получаем список слов
features = np.asarray(vectorizer.get_feature_names())

In [39]:
# находим слова с максимальным весом
answer = []
for i in max_weight:
    answer.append(features[i])
answer.sort()
answer

[u'atheism',
 u'atheists',
 u'bible',
 u'god',
 u'keith',
 u'moon',
 u'nick',
 u'religion',
 u'sky',
 u'space']